1. Bootstrapping (bagging - bootstrap aggregating):
 - техника выбора с повторениями
 - из общего пула данных берём отдельный объект, вносим его в выборку, формируем подвыборку из многих объектов, после чего кладём объекты назад
 - получается, случайно определяем количество подвыборок
 - случайно определяем размер каждой подвыборки (возможно при помощи random.randint), случайным образом выбираем элементы выборки и сохраняем в подвыборку
 - делаем все подвыборки, записываем их в список (List)

 - таким же образом поступаем со столбцами
 - определяем кол-во подвыборок столбцов
 - определяем размер каждого датафрейма (сколько столбцов он будет включать)
 - случайным образом определяем кол-во столбцов в каждой подвыборке
 - записываем подвыборки в список

 - записываем в список используемые модели 

 - прогоняем все выборки через каждую модель из списка

 - сохраняем результаты MSE для каждого прогона

 - создаем метод, в котором будет вычисляться значение predict для каждой модели и усредняться его значение

 - вычисляем общую MSE для ансамбля

 - сравниваем все MSE

In [2]:
from tqdm import tqdm
import warnings
warnings.simplefilter("ignore")

In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = 10, 6

In [15]:
from sklearn.datasets import load_digits as load
from sklearn.datasets import load_diabetes as diab
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.ensemble import BaggingClassifier, BaggingRegressor, RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [5]:
digits = load()
X = digits.data
y = digits.target

In [13]:
f = X.shape[1]

rnd_d3 = DecisionTreeClassifier(max_features=int(f ** 0.5))
d3 = DecisionTreeClassifier()

In [9]:
d3.fit(X, y)
d3.score(X, y)

1.0

In [11]:
print("Decision tree cross_val_score:", cross_val_score(d3, X, y).mean())

Decision tree cross_val_score: 0.7841318477251625


In [12]:
print("Bagging cross_val_score:", cross_val_score(BaggingClassifier(d3), X, y).mean())

Bagging cross_val_score: 0.8759455277004022


In [14]:
print("Randomized Bagging cross_val_score:", cross_val_score(BaggingClassifier(rnd_d3), X, y).mean())

Randomized Bagging cross_val_score: 0.9048638192510058


In [23]:
data = diab(as_frame=True).data
target = diab(as_frame=True).target

In [24]:
target

0      151.0
1       75.0
2      141.0
3      206.0
4      135.0
       ...  
437    178.0
438    104.0
439    132.0
440    220.0
441     57.0
Name: target, Length: 442, dtype: float64

In [25]:
data

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641
...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018114,0.044485
439,0.041708,0.050680,-0.015906,0.017293,-0.037344,-0.013840,-0.024993,-0.011080,-0.046883,0.015491
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044529,-0.025930


In [26]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.35)

In [27]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(287, 10) (155, 10) (287,) (155,)


In [38]:
rf = RandomForestRegressor(random_state=42, n_jobs=-2, oob_score=True, n_estimators=1000)

cv_results = cross_val_score(rf, X_train, y_train)

print(f"RF CV accuracy_score: {cv_results.mean():.3f}")
print(f"RF test accuracy_score: {rf.fit(X_train, y_train).score(X_test, y_test):.3f}")

RF CV accuracy_score: 0.311
RF test accuracy_score: 0.477


In [39]:
print(rf.predict(X_test))

[ 87.958 192.442 212.181 165.214 172.141  80.99  255.543  76.101  89.672
 217.787  88.3   242.735 195.143 121.955 132.702 221.811  90.369 183.024
  99.579 105.736 169.732 161.377 127.122 184.134 149.326 205.311 104.5
  83.531 128.409 141.499 208.884 116.236 168.273 242.562 134.29  224.586
 146.216 150.021 105.398 135.945 153.879  88.381  87.282 244.771 173.599
 212.957 145.732  78.729 177.035 108.683  91.083 104.521 170.246  99.383
 151.504  97.898  84.729  83.11   85.354 112.396 122.877 160.446 148.675
  93.946 173.42  155.12  113.291 112.846  91.932 152.723 165.882  90.201
 241.869 133.495 145.625 154.953 183.824 134.304 143.877 184.794  99.359
  89.208 208.444  93.032 115.16  177.636 243.711 149.939 254.084 189.431
 230.129 172.389 117.151 235.114 155.313 178.374 237.988 132.647 184.218
 208.522 251.798 143.201 224.408 103.812 258.944 160.575 119.841 166.948
 201.719  83.614 154.107 221.918 109.737 114.537 170.98   83.957 133.049
 174.092 104.755 230.355 168.223 240.597  92.133 114.

In [40]:
print(y_test)

294     55.0
205     99.0
139    281.0
180    107.0
169    152.0
       ...  
6      138.0
366    233.0
217    296.0
106    134.0
164    214.0
Name: target, Length: 155, dtype: float64


In [31]:
rf.oob_score_

0.013937282229965157

2. Blending

 - делим обучающую выборку на 2 подвыборки

 - обучаем несколько моделей на одной из подвыборок (KNN, Naive Bayes, SVM, LogisticRegression, DecisionTree, RandomForest)

 - Для 2-й подвыборки делаем предсказание каждой моделью значения таргета
 
 - У нас появилась новая обучающая выборка в новом признаковом пространстве (состоящая из предсказаний таргета каждой из моделей)
 
 - обучаем какую-нибудь другую модель (например взвешенная средняя) на новом датасете
 
 - откуда брать Ро - в лекции не сказано ()
 
 - получается на тестовой выборке делается predict и совокупность predict'ов скармливается в model.fit последней модели
 
 - и для предсказания нам нужно:
 
    а) сделать predict на всех моделях
    
    б) сделать predict из predict'ов на последней модели
    



3. Stacking

- заранее отделяем часть выборки для последнего тестирования

- делим оставшиеся данные на 3 фолда

- делим для каждого алгоритма - 2/3 обучение, 1/3 + тестовые данные - для предсказания, все комбинации

- получается - для всей обуч.выборки есть предсказания для 10 базовых алгоритмов и данные не потеряны

- после этого берем все наши модели, обучаем на всей модели полностью и предсказать метапризнаки для тестовой части выборки

- предсказанные 10-ю моделями данные являются метапризнаками 

- новая модель обучается на метапризнаках

- вероятно - также пропустить тестовый датасет через все 10 моделей для создания аналогичных метапризнаков и скормить их новой модели